In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df=pd.read_csv('T1.csv')

In [ ]:
df.drop(columns=df[['Date/Time','Theoretical_Power_Curve (KWh)']],inplace=True)
y=df['LV ActivePower (kW)']
df.drop(columns=['LV ActivePower (kW)'],axis=1,inplace=True)

In [ ]:
df['Wind Direction (°)']=(df['Wind Direction (°)']-df['Wind Direction (°)'].mean())/(df['Wind Direction (°)'].std())
df['Wind Speed (m/s)']=(df['Wind Speed (m/s)']-df['Wind Speed (m/s)'].mean())/(df['Wind Speed (m/s)'].std())

In [ ]:
df.head()

,Wind Speed (m/s),Wind Direction (°)
0,-0.531471,1.458710
1,-0.446111,1.551239
2,-0.554015,1.593229
3,-0.449066,1.579245
4,-0.468402,1.519489


In [ ]:
y_train=y[:42283] #data till october
y_test=y[42283:]
X_train=df.iloc[:42283]
X_test=df.iloc[42283:]

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score,r2_score,mean_squared_error

In [ ]:
xgr=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.7,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0.2,
             importance_type='gain', learning_rate=0.03, max_delta_step=0,
             max_depth=8, min_child_weight=25, missing=None, n_estimators=800,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0.2, reg_lambda=0.8, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.1, verbosity=1)

In [ ]:
sm=SVR(gamma='auto',C=50,epsilon=0.3)

In [ ]:
rf=RandomForestRegressor(n_estimators=500,max_depth=4)

In [ ]:
lr=LinearRegression()

In [ ]:
model=VotingRegressor([('lr',lr), ('rf',rf),('sm', sm),('xgr',xgr)],weights=[1,1,2,3])

In [ ]:
Model=model.fit(X_train, y_train)

[05:36:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
y_pred=Model.predict(X_test)
print(y_pred)
print('R2',r2_score(y_test,y_pred))
print('RMSE',np.sqrt(mean_squared_error(y_test,y_pred)))

[  37.19663285   35.11424591  166.48988105 ... 1495.99975726 2001.71956718
 2197.09806377]
R2 0.8966784593624547
RMSE 441.4402570054089


In [ ]:
from sklearn.model_selection import GridSearchCV

#max_depth = [None, 2,4,6,8,10,12]
#parameters = {"max_depth": max_depth}

In [ ]:
params={
 "learning_rate"    : [0.05, 0.01,0.03,0.1, 0.15, 0.2] ,
 "n_estimators"     : [50, 100, 150, 200, 500, 800,1000,1500] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15,20,25],
 "min_child_weight" : [ 1, 3, 5, 7 ,10,15,20,25],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "subsample"        : [ 0.1, 0.2 , 0.3, 0.4,0.6,0.8,1 ],
 "reg_lambda"       : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ,0.6,0.8,1],
 "reg_alpha"        : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7,0.9 ],
 "colsample_bylevel" : [ 0.3, 0.4, 0.5 , 0.7,0.9 ]
    
}

In [ ]:
df.head()

,Wind Speed (m/s),Wind Direction (°)
0,-0.531471,1.458710
1,-0.446111,1.551239
2,-0.554015,1.593229
3,-0.449066,1.579245
4,-0.468402,1.519489


In [ ]:
x=df
x.head()

,Wind Speed (m/s),Wind Direction (°)
0,-0.531471,1.458710
1,-0.446111,1.551239
2,-0.554015,1.593229
3,-0.449066,1.579245
4,-0.468402,1.519489


In [ ]:
y.head()

0    380.047791
1    453.769196
2    306.376587
3    419.645905
4    380.650696
Name: LV ActivePower (kW), dtype: float64

In [ ]:
regressor = VotingRegressor([('lr',lr), ('rf',rf),('sm', sm),('xgr',xgr)],weights=[1,1,2,3])
gs = GridSearchCV(estimator=regressor, param_grid=params, scoring='neg_mean_squared_error')
gs.fit(x,y)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/My Drive/predictions.csv')

In [ ]:
df.head()

,Unnamed: 0,windspeed,winddirection
0,0,2.974747,2.350521
1,1,2.378880,2.082964
2,2,2.539501,2.299246
3,3,2.760493,2.442138
4,4,2.471900,2.389178


In [ ]:
del df['Unnamed: 0']

In [ ]:
df.head()

,windspeed,winddirection
0,2.974747,2.350521
1,2.378880,2.082964
2,2.539501,2.299246
3,2.760493,2.442138
4,2.471900,2.389178


In [ ]:
df['windspeed']=(df['windspeed']-df['windspeed'].mean())/(df['windspeed'].std())
df['winddirection']=(df['winddirection']-df['winddirection'].mean())/(df['winddirection'].std())

In [ ]:
df.head()

,windspeed,winddirection
0,-1.024402,-1.214707
1,-1.283749,-1.330535
2,-1.213840,-1.236904
3,-1.117655,-1.175045
4,-1.243263,-1.197972


In [ ]:
df=df.rename(columns={"windspeed":"Wind Speed (m/s)"})


In [ ]:
df=df.rename(columns={"winddirection":"Wind Direction (°)"})

In [ ]:
df.head()

,Wind Speed (m/s),Wind Direction (°)
0,-1.024402,-1.214707
1,-1.283749,-1.330535
2,-1.213840,-1.236904
3,-1.117655,-1.175045
4,-1.243263,-1.197972


In [ ]:
y_pred=Model.predict(df)
print(y_pred)

[ 2.71891772e+01 -1.05232978e+02 -2.01280987e+00 -3.37584746e+01
 -4.02535698e+01 -1.33099543e+02 -1.09051415e+02 -8.09562169e+01
  2.19055147e+01 -4.10075004e+01  3.04453532e+02  2.09363415e+01
 -6.57545765e+01 -8.73198955e+01  1.15356520e+01  5.30413521e+01
  2.87670533e+02  3.79758574e+01 -1.48613097e+01 -2.59022257e+01
 -3.59916543e+01  2.19971069e+02  5.78692012e+02  5.03642798e+02
  3.36785567e+02  3.52594253e+02  3.59122678e+02  5.81720891e+02
  1.07748648e+03  1.57217558e+03  1.56527764e+03  1.74845106e+03
  1.57621418e+03  1.23178234e+03  1.68455844e+03  3.12061773e+03
  3.34095713e+03  3.30901950e+03  3.40474266e+03  3.27694509e+03
  1.44189162e+03  7.57788039e+02  6.09657348e+02  7.31838101e+02
  5.62992794e+02  6.20347482e+02  5.78978740e+02  5.75894792e+02
  5.96845264e+02  7.33911025e+02  7.30454623e+02  7.71972141e+02
  8.64486015e+02  1.03174664e+03  1.46006982e+03  1.22065482e+03
  1.79534687e+03  3.27489626e+03  3.35020046e+03  3.42008080e+03
  3.31397815e+03  2.99333

In [ ]:
import pickle

In [ ]:
data = {"model": Model, "windspeed": 'Wind Speed (m/s)', "winddirection": 'Wind Direction (°)'}
with open('saved_model.pkl', 'wb') as file:
    pickle.dump(data, file)

In [ ]:
with open('saved_model.pkl', 'rb') as file:
    data = pickle.load(file)

model_loaded = data["model"]
windspeed = data["windspeed"]
winddirection = data["winddirection"]

[05:44:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
y_pred = model_loaded.predict(df)

In [ ]:
print(y_pred)

[ 2.71891772e+01 -1.05232978e+02 -2.01280987e+00 -3.37584746e+01
 -4.02535698e+01 -1.33099543e+02 -1.09051415e+02 -8.09562169e+01
  2.19055147e+01 -4.10075004e+01  3.04453532e+02  2.09363415e+01
 -6.57545765e+01 -8.73198955e+01  1.15356520e+01  5.30413521e+01
  2.87670533e+02  3.79758574e+01 -1.48613097e+01 -2.59022257e+01
 -3.59916543e+01  2.19971069e+02  5.78692012e+02  5.03642798e+02
  3.36785567e+02  3.52594253e+02  3.59122678e+02  5.81720891e+02
  1.07748648e+03  1.57217558e+03  1.56527764e+03  1.74845106e+03
  1.57621418e+03  1.23178234e+03  1.68455844e+03  3.12061773e+03
  3.34095713e+03  3.30901950e+03  3.40474266e+03  3.27694509e+03
  1.44189162e+03  7.57788039e+02  6.09657348e+02  7.31838101e+02
  5.62992794e+02  6.20347482e+02  5.78978740e+02  5.75894792e+02
  5.96845264e+02  7.33911025e+02  7.30454623e+02  7.71972141e+02
  8.64486015e+02  1.03174664e+03  1.46006982e+03  1.22065482e+03
  1.79534687e+03  3.27489626e+03  3.35020046e+03  3.42008080e+03
  3.31397815e+03  2.99333